In [ ]:
#Jaderná elektrárna Dukovany(두코바니 원자력 발전소)

#https://search.seznam.cz/clanky/?q=Jadern%C3%A1%20elektr%C3%A1rna%20Dukovany
#리눅스 환경에서 코드 실행 중

In [ ]:
import time
import json
import datetime
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
from webdriver_manager.chrome import ChromeDriverManager

# 🔍 검색어 리스트
search_queries = ["두코바니 원전"]  

# ✅ 크롤링할 기간 설정 (2022.01.01 ~ 2024.12.31)
START_DATE = datetime.datetime(2022, 1, 1)
END_DATE = datetime.datetime(2024, 12, 31)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # GUI 없이 실행
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# ✅ 네이버 뉴스 검색 URL 템플릿 (날짜별로 검색)
NAVER_NEWS_URL_TEMPLATE = (
    "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0"
    "&field=0&pd=3&ds={start_date}&de={end_date}"
)

def fetch_naver_news(query, start_date, end_date):
    """Selenium을 사용하여 네이버 뉴스 검색 페이지 크롤링 (무한 스크롤)"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query, start_date=start_date, end_date=end_date)
    driver.get(search_url)
    time.sleep(2)  # 페이지 로딩 대기

    # ✅ 무한 스크롤을 통해 뉴스 목록 로드
    print(f"🔍 검색 중: {query} ({start_date} ~ {end_date})")
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  
        last_height = new_height

    # ✅ BeautifulSoup으로 페이지 파싱
    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            # ✅ 날짜 가져오기 (목록에서 크롤링)
            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 링크 및 날짜를 수집 완료! ({start_date} ~ {end_date})")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        try:
            response = requests.get(news["link"], headers=HEADERS)
            soup = BeautifulSoup(response.text, "html.parser")

            # ✅ 본문 크롤링 (다양한 선택자 추가)
            content_selectors = [
                "div#newsct_article", "div#articleBodyContents", "div.article_view",
                "div.news_end", "div#articeBody", "div.content_area",
                "div#newsEndContents", "article"
            ]
            content = ""
            for selector in content_selectors:
                content_element = soup.select(selector)
                if content_element:
                    content = " ".join([p.text.strip() for p in content_element])
                    break

            # ✅ Selenium을 사용한 백업 크롤링
            if not content.strip():
                print(f"⚠ 본문을 가져오지 못해 Selenium으로 재시도: {news['link']}")
                driver.get(news["link"])
                time.sleep(2)
                soup = BeautifulSoup(driver.page_source, "html.parser")
                content_element = soup.select_one("div#newsct_article") or soup.select_one("article")
                if content_element:
                    content = " ".join([p.text.strip() for p in content_element])

            # ✅ 데이터 저장
            news["content"] = content.strip() if content else "N/A"
            news_data.append(news)

            print(f"✅ 크롤링 완료: {news['title'][:50]}...")

            time.sleep(1)

        except Exception as e:
            print(f"⚠ 오류 발생 (뉴스 크롤링 실패): {news['link']}, 오류: {e}")

    return news_data

# ✅ 날짜 범위를 3개월씩 쪼개서 크롤링 (2000개 초과 방지)
all_news = []
date_ranges = [
    ("2022.01.01", "2022.03.31"), ("2022.04.01", "2022.06.30"),
    ("2022.07.01", "2022.09.30"), ("2022.10.01", "2022.12.31"),
    ("2023.01.01", "2023.03.31"), ("2023.04.01", "2023.06.30"),
    ("2023.07.01", "2023.09.30"), ("2023.10.01", "2023.12.31"),
    ("2024.01.01", "2024.03.31"), ("2024.04.01", "2024.06.30"),
    ("2024.07.01", "2024.09.30"), ("2024.10.01", "2024.12.31")
]

for query in search_queries:
    for start_date, end_date in date_ranges:
        news_list = fetch_naver_news(query, start_date, end_date)
        news_data = fetch_news_content(news_list)
        all_news.extend(news_data)

# ✅ JSON 파일로 저장
output_file = "naver_news.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(all_news, f, ensure_ascii=False, indent=4)

print(f"✅ 총 {len(all_news)}개의 뉴스 기사를 저장했습니다. (파일명: {output_file})")

# ✅ Selenium 종료
driver.quit()


🔍 검색 중: 두코바니 원전 (2022.01.01 ~ 2022.03.31)
✅ 198개의 뉴스 기사 링크 및 날짜를 수집 완료! (2022.01.01 ~ 2022.03.31)


📰 뉴스 본문 크롤링 진행:   0%|          | 0/198 [00:00<?, ?it/s]

⚠ 본문을 가져오지 못해 Selenium으로 재시도: http://www.edaily.co.kr/news/newspath.asp?newsid=02273046632268896
✅ 크롤링 완료: 산업부, 체코와의 공동 원자력 연구사업에 12억원 지원...


📰 뉴스 본문 크롤링 진행:   1%|          | 1/198 [00:06<21:05,  6.42s/it]

⚠ 본문을 가져오지 못해 Selenium으로 재시도: http://www.edaily.co.kr/news/newspath.asp?newsid=03863846632268896
✅ 크롤링 완료: 체코 원전사업 수주 ‘드림팀’ 띄우나…인수위, 윤석열 당선인에 보고...


...

✅ 크롤링 완료: 황주호 한수원 사장, 체코 신규원전 세일즈 나서...


📰 뉴스 본문 크롤링 진행:  73%|███████▎  | 122/167 [06:24<03:33,  4.75s/it]

✅ 크롤링 완료: [시론]2024년은 원전 산업계 도약 기반 마련의 시간...


📰 뉴스 본문 크롤링 진행:  74%|███████▎  | 123/167 [06:25<02:43,  3.71s/it]

⚠ 본문을 가져오지 못해 Selenium으로 재시도: https://biz.sbs.co.kr/article_hub/20000155313?division=NAVER
